In [1]:
import os 
import pandas as pd
print("modules imported")

# establish paths
domain = "Rockies"
outputWorkspace = f"D:/ASOML/{domain}/modelOutputs/fromAlpine/"

model_runs = ["20250606_153243", "20250606_165425", "20250606_213944", "20250606_220629", "20250606_233029", "20250606_235052",
       "20250607_070022", "20250607_083849", "20250607_211913", "20250607_220914", "20250608_080730", "20250608_092651", "20250612_134159",
             "20250613_051259", "20250613_065322", "20250709_092529", "20250709_061842", "20250708_114938"]
groups = ["G1", "G2", "G3", "G4", "G5", "G6"]

best_by_group = {}  
score_column = "RMSE"
group_column = "GroupNum"

for run in model_runs:
    folder = f"{outputWorkspace}{run}/"
    errorCSV = f"{folder}/errorReview/{run}_error_summary_stats.csv"

    if os.path.exists(errorCSV):
        try:
            df = pd.read_csv(errorCSV)

            for _, row in df.iterrows():
                group = row[group_column]
                score = row[score_column]

                # Check for valid numeric score
                if pd.notna(score):
                    if group not in best_by_group or score < best_by_group[group]['score']:
                        best_by_group[group] = {
                            'score': score,
                            'model_run': run,
                            'folder': folder,
                            'row': row
                        }

        except Exception as e:
            print(f"Error reading {errorCSV}: {e}")
    else:
        print(f"CSV not found for model run: {run}")

# ✅ Report results
print("\n Best RMSE by Group:")
for group, result in best_by_group.items():
    print(f"\n Group: {group}")
    print(f"Model Run: {result['model_run']}")
    print(f"RMSE: {result['score']}")
    print(f"Full row:\n{result['row']}")


modules imported

 Best RMSE by Group:

 Group: G1
Model Run: 20250612_134159
RMSE: 0.0626340458145326
Full row:
GroupNum            G1
basin       ClearCreek
doy            2025099
year              2025
RMSE          0.062634
MSE           0.003923
MAE           0.037329
n_valid         283031
Name: 0, dtype: object

 Group: G2
Model Run: 20250606_213944
RMSE: 0.0968406788183262
Full row:
GroupNum           G2
basin       BlueRiver
doy           2025101
year             2025
RMSE         0.096841
MSE          0.009378
MAE          0.067898
n_valid        231702
Name: 1, dtype: object

 Group: G3
Model Run: 20250709_092529
RMSE: 0.1298893865363854
Full row:
GroupNum               G3
basin       GunnisonNorth
doy               2025117
year                 2025
RMSE             0.129889
MSE              0.016871
MAE              0.061684
n_valid            376916
Name: 2, dtype: object

 Group: G4
Model Run: 20250607_070022
RMSE: 0.0726703173613838
Full row:
GroupNum                G4
b